# **Step 1: Prepare the dataset**

a . Get the images of Indian Cars with number plate.
Create 2 folders 'test' and 'train' and trandfer 30% of the images in test and 70% images in train folders respectively.

Alternatively, you can use the test and train data used by me for training.
test and train folders with annotation files are attached to this repository.


These activities should to be performed manually. For more details please refer the article link -  


b. Label the licence plates in all the images in respective test and train folders, using LabelImg and generate the annoted .xml file for the images within the respective folders.
You can get more details on LabelImg from [here](https://github.com/tzutalin/labelImg)

# **Step 2: Prepare the dataset**

Download YOLOv3 project

`! git clone https://github.com/AlexeyAB/darknet`


Open darknet/Makefile and put 1 infront of GPU, CUDNN and OPENCV, instead of 0.
 
These changes are required if you wanted to use GPU on google colab.

# **Step 3: Installing the base darknet system**

Install the base darknet framework with the below commands.

```
! cd darknet
! make
```



# **Step 4: Upload images in Google Colab and generate YOLO compatible annotation files.**


a. Mount the Google Drive
```
from google.colab import drive
drive.mount('/content/gdrive')

cd /content/gdrive/MyDrive
```



b. Upload test and train folders in darknet folder within Google Colab. (For this step, use images present in the test and train folders) 

c. To convert the bounding box coordinates from the .xml file to the .txt file(YOLO compatible) run the following commands.

```
! cd darknet
! python convert.py
```

This step will pick the coordinates from the .xml file and put them into YOLO compatible .txt file in the same directories as images. Also, train.txt and test.txt files are created within the darknet folder, containing the location of images.

# **Step 5: Prepare custom 'my_data' folder**

a. Run the below command to make a custome folder named 'my_data'
```
! mkdir my_data
```
b. Move the train.txt and test.txt files from the darknet directory to my_data directory.
```
! mv train.txt my_data/
! mv test.txt my_data/
```
c. Create classes.names file within my_data directory with class name as "LP".
```
! touch darknet/my_data/classes.names
! echo LP > classes.names
```
d. Create weights directory within my_data directory.
```
! mkdir my_data/weights
```
e. Create file darknet.data within my_data directory to provide the configuration details.
```
! touch darknet/my_data/darknet.data
```

Paste the below details in darknet.data file.
*   classes = 1
*   train  = my_data/train.txt
*   valid  = my_data/test.txt
*   names = my_data/classes.names
*   backup = my_data/weights/

f. Copy and Paste the cfg file from darknet/cfg/yolov3.cfg to darknet/my_data directory.
```
! cp   darknet/cfg/yolov3.cfg to darknet/my_data
```

Make the following changes in yolov3.cfg in my_data directory
*   Line 603, 693, and 780 change the filters to 18. (filters = (classes + 5) * 3). In our case we are detecting only 1 class, so the number of filters will be equal to 18.

*   Line 783, change the number of classes to 1.




# **Step 6: Download the initial yolo weights for training the custom data**


Run the following command from the darknet directory.
```
! wget https://pjreddie.com/media/files/darknet53.conv.74
```



# **Step 7: Set criteria to save the weights file in weights directory**

Open detector.c file from darknet/examples directory and change line number 138 as shown below.
```
if(i%1000==0 || (i < 1000 && i%200 == 0))
```
This change saves the weight in my_data/weights directory for every 200th iteration till 1000 iterations and then for every 1000th iteration. For the demonstration, I have trained this model for 8000 iterations only.


# **Step 8: Now start the training**

Run the follwoing command from the darknet directory.
```
! ./darknet detector train my_data/darknet.data my_data/yolov3.cfg darknet53.conv.74
```
Note: In case you get "/bin/bash: ./darknet: Permission denied" error then run the below command and then run the above command.
```
! chmod +x darknet
```

Note: 
You might not be able to train the model in one shot due to restictions in google colab.
Check the lastest weight stored in my_data/weights folder and then train the model again with the updated weights. In case you have yolov3_8000.weights in your weights folder then run the below command to train the model again with new weights.
```
! ./darknet detector train my_data/darknet.data my_data/yolov3.cfg /content/gdrive/MyDrive/darknet/my_data/weights/yolov3_8000.weights
```



# **Step 8: Detect and Blur the Licence Plate from the image**

Change the weightsPath in line number 12 and the image file name in line number 15 before running the below code.

You should see the image of a car with the detection and blurred number plate.



```
%matplotlib inline
import numpy as np
import imutils
import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

CONF_THRESH, NMS_THRESH = 0.5, 0.5

weightsPath = 'my_data/weights/yolov3_8000.weights'
configPath = 'my_data/yolov3.cfg'
namesPath = 'my_data/classes.names'
image = 'images8.jpg'

# Load the network using openCV
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

# Get the output layer from YOLOv3
layers = net.getLayerNames()
output_layers = [layers[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Read and convert the image to blob and perform forward pass
img = cv2.imread(image)
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
height, width = img.shape[:2]

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), swapRB=True, crop=False)
net.setInput(blob)
layer_outputs = net.forward(output_layers)

class_ids, confidences, b_boxes = [], [], []
for output in layer_outputs:
for detection in output:
scores = detection[5:]
class_id = np.argmax(scores)
confidence = scores[class_id]

if confidence > CONF_THRESH:
center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')

x = int(center_x - w / 2)
y = int(center_y - h / 2)

b_boxes.append([x, y, int(w), int(h)])
confidences.append(float(confidence))
class_ids.append(int(class_id))

# Perform non maximum suppression for the bounding boxes to filter overlapping and low confident bounding boxes
indices = cv2.dnn.NMSBoxes(b_boxes, confidences, CONF_THRESH, NMS_THRESH).flatten().tolist()

if len(indices) > 0:

# Draw the filtered bounding boxes with their class to the image
with open(namesPath, "r") as f:
classes = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for index in indices:
(x,y) = (b_boxes[index][0], b_boxes[index][1])
(w,h) = (b_boxes[index][2], b_boxes[index][3])
# Blur the ROI of the detected licence plate 
img[y:y+h, x:x+w] = cv2.GaussianBlur(img[y:y+h, x:x+w] ,(35,35),0)

cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
text = "{}: {:.4f}".format("LP", confidences[index])
cv2.putText(img, text, (x, y - 3), cv2.FONT_HERSHEY_COMPLEX_SMALL, .75 , (0, 255, 0), 1)

plt.figure(figsize=(10, 5))
plt.imshow(img)
plt.show()
```